In [1]:
from math import sqrt
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, LSTM
from numpy import concatenate
from statistics import mean
import random
import matplotlib.pyplot as plt

Using TensorFlow backend.
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/alexellard/opt/anacond

In [2]:
df = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
df

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [ ]:
#average scores flows link 1

MAE_average = []
RMSE_average = []
R2_average = []
maximum_MAE = []
sensors_covered = []
n_columns_to_drop = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
for n in n_columns_to_drop:
    MAE = []
    RMSE = []
    R2 = []
    for run in range(1,11):
        Y = df[df.columns[16]]
        X = df.drop(columns=['node_16_demand'])
        X['node_16_demand'] = Y
        columns = []
        columns_to_drop = random.sample(list(X.columns), n)
        # Drop the selected columns from the dataframe
        if X.columns[-1] in columns_to_drop:
            columns_to_drop.remove(X.columns[-1])
        X.drop(columns_to_drop, axis=1, inplace=True)
        columns.append(columns_to_drop)
        print(columns)
        print('starting df shape:', X.shape)
        print(f'This is training run {run} and has the shape: ', X.shape)
        print(f'The value of dropped columns is {n}')

        # split into train and test sets
        values = X.values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        train, test = train_test_split(scaled, test_size=0.2, random_state=0)

        # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]

        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
        print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        # fit network
        model.fit(train_X, train_y, epochs=10, batch_size=128, validation_data=(test_X, test_y), verbose=0, shuffle=False)

        # make predictions on the test data
        y_pred = model.predict(test_X)
        test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

        test_y = test_y.reshape((len(test_y), 1))
        print('test y shape :', test_y.shape)
        inv_test_y = concatenate((test_y, test_X[:, -X.shape[1]:]), axis=1)
        print('Inv y shape :', inv_test_y.shape)
        test_y_inverse = scaler.inverse_transform(inv_test_y)
        test_y_inverse = test_y_inverse[:,0]

        y_pred = y_pred.reshape((len(y_pred), 1))
        print('y pred shape :', y_pred.shape)
        inv_y_pred = concatenate((y_pred, test_X[:, -X.shape[1]:]), axis=1)
        print('inv y pred :', inv_y_pred.shape)
        y_pred_inverse = scaler.inverse_transform(inv_y_pred)
        y_pred_inverse = y_pred_inverse[:,0]

        # evaluate scaled errors
        mae_inverse = mean_absolute_error(test_y_inverse, y_pred_inverse)
        MAE.append(mae_inverse)
        print(f'Run {run} Mean Absolute Error:', mae_inverse)
        rmse_inverse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
        RMSE.append(rmse_inverse)
        print('RMSE: %.3f' % rmse_inverse)
        r2_inverse = r2_score(test_y_inverse, y_pred_inverse)
        R2.append(r2_inverse)
        print('R^2: %.3f' % r2_inverse)
    percentage_list = round(X.shape[1] / 96 * 100)
    sensors_covered.append(percentage_list)
    MAE_mean = mean(MAE)
    RMSE_mean = mean(RMSE)
    R2_mean = mean(R2)
    max_MAE = max(MAE)
    maximum_MAE.append(max_MAE)
    MAE_average.append(MAE_mean)
    RMSE_average.append(RMSE_mean)
    R2_average.append(R2_mean)
print('Length MAE_average: ', len(MAE_average))
print('Length sensors_covered: ', len(sensors_covered))

[[]]
starting df shape: (87600, 96)
This is training run 1 and has the shape:  (87600, 96)
The value of dropped columns is 0
(70080, 1, 95) (70080,) (17520, 1, 95) (17520,)

test y shape : (17520, 1)
Inv y shape : (17520, 96)
y pred shape : (17520, 1)
inv y pred : (17520, 96)
Run 1 Mean Absolute Error: 68.89612044147391
RMSE: 84.345
R^2: 0.997
[[]]
starting df shape: (87600, 96)
This is training run 2 and has the shape:  (87600, 96)
The value of dropped columns is 0
(70080, 1, 95) (70080,) (17520, 1, 95) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 96)
y pred shape : (17520, 1)
inv y pred : (17520, 96)
Run 2 Mean Absolute Error: 63.65691494320769
RMSE: 79.194
R^2: 0.997
[[]]
starting df shape: (87600, 96)
This is training run 3 and has the shape:  (87600, 96)
The value of dropped columns is 0
(70080, 1, 95) (70080,) (17520, 1, 95) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 96)
y pred shape : (17520, 1)
inv y pred : (17520, 96)
Run 3 Mean Absolute Error: 56.461

test y shape : (17520, 1)
Inv y shape : (17520, 86)
y pred shape : (17520, 1)
inv y pred : (17520, 86)
Run 1 Mean Absolute Error: 116.03495406458926
RMSE: 130.362
R^2: 0.993
[['node_9_demand', 'node_25_pressure', 'node_28_pressure', 'link_22_flow', 'link_24_flow', 'link_4_flow', 'node_24_demand', 'node_4_pressure', 'node_7_demand', 'link_27_flow']]
starting df shape: (87600, 86)
This is training run 2 and has the shape:  (87600, 86)
The value of dropped columns is 10
(70080, 1, 85) (70080,) (17520, 1, 85) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 86)
y pred shape : (17520, 1)
inv y pred : (17520, 86)
Run 2 Mean Absolute Error: 51.41639512377366
RMSE: 64.719
R^2: 0.998
[['link_27_flow', 'node_8_demand', 'node_10_demand', 'node_4_demand', 'link_16_flow', 'node_18_pressure', 'node_10_pressure', 'node_7_pressure', 'node_3_demand']]
starting df shape: (87600, 87)
This is training run 3 and has the shape:  (87600, 87)
The value of dropped columns is 10
(70080, 1, 86) (70080,) 

(70080, 1, 80) (70080,) (17520, 1, 80) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 81)
y pred shape : (17520, 1)
inv y pred : (17520, 81)
Run 6 Mean Absolute Error: 96.23750859469085
RMSE: 120.526
R^2: 0.994
[['node_24_demand', 'link_14_flow', 'node_24_pressure', 'link_25_flow', 'link_20_flow', 'link_24_flow', 'node_1_pressure', 'node_18_pressure', 'node_10_pressure', 'node_2_demand', 'node_8_pressure', 'node_3_demand', 'node_26_pressure', 'node_25_demand', 'node_4_pressure']]
starting df shape: (87600, 81)
This is training run 7 and has the shape:  (87600, 81)
The value of dropped columns is 15
(70080, 1, 80) (70080,) (17520, 1, 80) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 81)
y pred shape : (17520, 1)
inv y pred : (17520, 81)
Run 7 Mean Absolute Error: 130.22856217115722
RMSE: 144.844
R^2: 0.991
[['node_19_pressure', 'node_18_pressure', 'link_12_flow', 'node_19_demand', 'node_7_pressure', 'node_12_demand', 'link_30_flow', 'link_32_flow', 'link_9_flow', 'n

[['node_28_demand', 'node_30_pressure', 'node_27_pressure', 'node_8_demand', 'link_25_flow', 'node_17_pressure', 'link_13_flow', 'node_15_demand', 'node_23_demand', 'node_30_demand', 'node_1_demand', 'node_19_pressure', 'link_18_flow', 'link_27_flow', 'node_8_pressure', 'node_24_pressure', 'reservoir_demand', 'node_3_demand', 'node_10_pressure', 'link_17_flow']]
starting df shape: (87600, 76)
This is training run 9 and has the shape:  (87600, 76)
The value of dropped columns is 20
(70080, 1, 75) (70080,) (17520, 1, 75) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 76)
y pred shape : (17520, 1)
inv y pred : (17520, 76)
Run 9 Mean Absolute Error: 2.5644193765694845
RMSE: 3.302
R^2: 0.998
[['node_26_pressure', 'link_32_flow', 'node_22_demand', 'link_24_flow', 'node_21_demand', 'node_30_demand', 'node_2_demand', 'node_26_demand', 'link_18_flow', 'link_14_flow', 'node_28_pressure', 'node_1_pressure', 'link_4_flow', 'node_8_pressure', 'link_11_flow', 'node_4_demand', 'link_5_flow'

(70080, 1, 70) (70080,) (17520, 1, 70) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 71)
y pred shape : (17520, 1)
inv y pred : (17520, 71)
Run 9 Mean Absolute Error: 47.76629520842422
RMSE: 60.621
R^2: 0.998
[['link_24_flow', 'link_19_flow', 'node_27_demand', 'node_4_demand', 'node_12_pressure', 'node_21_demand', 'node_7_demand', 'node_5_pressure', 'node_11_demand', 'node_20_pressure', 'node_30_demand', 'link_10_flow', 'node_29_pressure', 'link_15_flow', 'node_10_pressure', 'node_28_demand', 'node_14_demand', 'node_5_demand', 'node_18_demand', 'node_30_pressure', 'node_22_pressure', 'node_31_demand', 'node_7_pressure', 'link_7_flow', 'link_21_flow']]
starting df shape: (87600, 71)
This is training run 10 and has the shape:  (87600, 71)
The value of dropped columns is 25
(70080, 1, 70) (70080,) (17520, 1, 70) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 71)
y pred shape : (17520, 1)
inv y pred : (17520, 71)
Run 10 Mean Absolute Error: 56.5245182634224
RMSE: 71.44

(70080, 1, 66) (70080,) (17520, 1, 66) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 67)
y pred shape : (17520, 1)
inv y pred : (17520, 67)
Run 9 Mean Absolute Error: 55.046727306893494
RMSE: 68.814
R^2: 0.998
[['node_23_pressure', 'node_9_demand', 'link_18_flow', 'node_23_demand', 'node_14_demand', 'link_6_flow', 'node_3_pressure', 'node_20_demand', 'node_31_demand', 'link_11_flow', 'link_23_flow', 'reservoir_pressure', 'link_12_flow', 'node_17_pressure', 'link_1_flow', 'link_7_flow', 'node_6_pressure', 'node_2_demand', 'link_24_flow', 'node_8_pressure', 'node_4_demand', 'link_26_flow', 'reservoir_demand', 'node_10_demand', 'node_15_demand', 'node_20_pressure', 'link_25_flow', 'node_18_demand', 'node_24_pressure']]
starting df shape: (87600, 67)
This is training run 10 and has the shape:  (87600, 67)
The value of dropped columns is 30
(70080, 1, 66) (70080,) (17520, 1, 66) (17520,)
test y shape : (17520, 1)
Inv y shape : (17520, 67)
y pred shape : (17520, 1)
inv y pred : (1

test y shape : (17520, 1)
Inv y shape : (17520, 62)
y pred shape : (17520, 1)
inv y pred : (17520, 62)
Run 8 Mean Absolute Error: 7.813653175985111
RMSE: 10.158
R^2: 0.985
[['node_30_pressure', 'node_1_demand', 'node_20_pressure', 'node_1_pressure', 'link_25_flow', 'link_19_flow', 'link_21_flow', 'node_9_pressure', 'node_8_demand', 'link_31_flow', 'node_24_pressure', 'link_32_flow', 'node_7_pressure', 'link_14_flow', 'link_20_flow', 'node_10_pressure', 'link_30_flow', 'node_29_pressure', 'node_27_pressure', 'node_18_pressure', 'link_27_flow', 'link_16_flow', 'node_7_demand', 'link_28_flow', 'node_15_demand', 'node_14_pressure', 'link_1_flow', 'link_24_flow', 'node_19_demand', 'link_22_flow', 'node_28_demand', 'node_26_pressure', 'link_2_flow', 'node_4_pressure', 'link_26_flow']]
starting df shape: (87600, 61)
This is training run 9 and has the shape:  (87600, 61)
The value of dropped columns is 35
(70080, 1, 60) (70080,) (17520, 1, 60) (17520,)
test y shape : (17520, 1)
Inv y shape : (

test y shape : (17520, 1)
Inv y shape : (17520, 56)
y pred shape : (17520, 1)
inv y pred : (17520, 56)
Run 6 Mean Absolute Error: 65.97113993649323
RMSE: 80.162
R^2: 0.997
[['node_13_pressure', 'node_10_demand', 'link_29_flow', 'node_27_pressure', 'node_28_demand', 'link_28_flow', 'node_1_pressure', 'link_19_flow', 'node_2_pressure', 'link_32_flow', 'node_25_demand', 'node_15_demand', 'node_4_pressure', 'node_2_demand', 'node_12_pressure', 'node_24_demand', 'node_29_demand', 'node_28_pressure', 'node_1_demand', 'link_24_flow', 'node_20_pressure', 'link_4_flow', 'node_29_pressure', 'node_7_pressure', 'link_14_flow', 'node_18_demand', 'node_31_demand', 'link_11_flow', 'node_21_demand', 'link_8_flow', 'node_11_pressure', 'node_7_demand', 'node_6_demand', 'node_15_pressure', 'node_10_pressure', 'node_3_demand', 'node_13_demand', 'node_8_pressure', 'node_5_demand', 'link_22_flow']]
starting df shape: (87600, 56)
This is training run 7 and has the shape:  (87600, 56)
The value of dropped col